# Block Aligner Benchmark Analysis and Visualizations

This notebook contains code for collecting, cleaning, and analyzing data produced by block aligner's experiments.

To run this, you will need to install all the libraries imported below, along with [altair-saver](https://github.com/altair-viz/altair_saver) and [altair-data-server](https://github.com/altair-viz/altair_data_server), which has some extra dependencies for PDF saving.

Run each cell one by one to reproduce the experiments. This may take a while. For accurate benchmarking, it is recommended to run the entire notebook in the command line with `nbconvert`.

In [1]:
import altair as alt
from altair_saver import save
from altair import datum
import pandas as pd
from io import StringIO

alt.data_transformers.enable("data_server")

DataTransformerRegistry.enable('data_server')

In [2]:
def csv_to_pandas(csv, d = "\\s*,\\s*", t = None):
    s = StringIO("\n".join(csv))
    data = pd.read_csv(s, sep = d, thousands = t, comment = "#", engine = "python")
    return data

## Prefix Scan Benchmark

In [3]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- prefix_scan | grep 'bench:' | awk '{print $2"\t"$5}'
output.insert(0, "algorithm\ttime")
output

['algorithm\ttime', 'bench_naive_prefix_scan\t26', 'bench_opt_prefix_scan\t18']

In [4]:
data = csv_to_pandas(output, d = "\t", t = ",")
data

,algorithm,time
0,bench_naive_prefix_scan,26
1,bench_opt_prefix_scan,18


In [5]:
data["algorithm"] = data["algorithm"].map({
    "bench_naive_prefix_scan": "naive",
    "bench_opt_prefix_scan": "ours"
})
data

,algorithm,time
0,naive,26
1,ours,18


Prefix Scan Benchmark (AVX2)

In [6]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("time", axis = alt.Axis(title = "time (ns)")),
    y = "algorithm",
    color = alt.Color("algorithm", legend = None)
).properties(
    width = 150
)
save(c, "prefix_scan_bench.pdf")
c

alt.Chart(...)

## Random Data Benchmark

In [7]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- bench_ | grep 'bench:' | grep -v 'prefix_scan' | awk '{print $2"\t"$5}'
output

['bench_parasailors_aa_1000_10000\t51,466,626',
 'bench_parasailors_aa_100_1000\t617,200',
 'bench_parasailors_aa_10_100\t20,655',
 'bench_rustbio_aa_100_1000\t15,360,650',
 'bench_rustbio_aa_10_100\t163,398',
 'bench_scan_aa_1000_10000\t227,082',
 'bench_scan_aa_1000_10000_insert\t4,750,883',
 'bench_scan_aa_1000_10000_small\t214,008',
 'bench_scan_aa_1000_10000_trace\t1,881,374',
 'bench_scan_aa_100_1000\t23,215',
 'bench_scan_aa_100_1000_insert\t50,134',
 'bench_scan_aa_100_1000_small\t22,462',
 'bench_scan_aa_100_1000_trace\t520,633',
 'bench_scan_aa_10_100\t3,847',
 'bench_scan_aa_10_100_insert\t4,003',
 'bench_scan_aa_10_100_small\t3,369',
 'bench_scan_aa_10_100_trace\t364,496',
 'bench_scan_nuc_1000_10000\t215,741',
 'bench_scan_nuc_100_1000\t22,209',
 'bench_triple_accel_1000_10000\t8,068,266',
 'bench_triple_accel_100_1000\t23,737']

In [8]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["parasailors_aa", "rustbio_aa", "scan_aa", "scan_nuc", "triple_accel"]
new_names = ["parasailors\tprotein", "rust bio\tprotein", "ours\tprotein", "ours\tnucleotide", "triple accel\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'parasailors\tprotein\t1000\t10000\tdefault\t51,466,626',
 'parasailors\tprotein\t100\t1000\tdefault\t617,200',
 'parasailors\tprotein\t10\t100\tdefault\t20,655',
 'rust bio\tprotein\t100\t1000\tdefault\t15,360,650',
 'rust bio\tprotein\t10\t100\tdefault\t163,398',
 'ours\tprotein\t1000\t10000\tdefault\t227,082',
 'ours\tprotein\t1000\t10000\tinsert\t4,750,883',
 'ours\tprotein\t1000\t10000\tsmall\t214,008',
 'ours\tprotein\t1000\t10000\ttrace\t1,881,374',
 'ours\tprotein\t100\t1000\tdefault\t23,215',
 'ours\tprotein\t100\t1000\tinsert\t50,134',
 'ours\tprotein\t100\t1000\tsmall\t22,462',
 'ours\tprotein\t100\t1000\ttrace\t520,633',
 'ours\tprotein\t10\t100\tdefault\t3,847',
 'ours\tprotein\t10\t100\tinsert\t4,003',
 'ours\tprotein\t10\t100\tsmall\t3,369',
 'ours\tprotein\t10\t100\ttrace\t364,496',
 'ours\tnucleotide\t1000\t10000\tdefault\t215,741',
 'ours\tnucleotide\t100\t1000\tdefault\t22,209',
 'triple accel\tnucleotide\t1000\t100

In [9]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,parasailors,protein,1000,10000,default,51466626
1,parasailors,protein,100,1000,default,617200
2,parasailors,protein,10,100,default,20655
3,rust bio,protein,100,1000,default,15360650
4,rust bio,protein,10,100,default,163398
5,ours,protein,1000,10000,default,227082
6,ours,protein,1000,10000,insert,4750883
7,ours,protein,1000,10000,small,214008
8,ours,protein,1000,10000,trace,1881374
9,ours,protein,100,1000,default,23215


In [10]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (AVX2)

In [11]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench.pdf")
c

alt.Chart(...)

Random DNA Sequences Benchmark (AVX2)

In [12]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000])),
    shape = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000]))
).transform_filter(
    datum.alphabet == "nucleotide"
).properties(
    width = 200,
    height = 50
)
save(c, "random_dna_bench.pdf")
c

alt.Chart(...)

## Uniclust 30 Data Benchmark

In [13]:
output = !cd .. && cargo run --example uc_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, size, time',
 'ours (no trace), uc30, 32-32, 0.057246475',
 'ours (no trace), uc30 0.95, 32-32, 0.063216217',
 'ours (no trace), uc30, 32-256, 0.09164253',
 'ours (no trace), uc30 0.95, 32-256, 0.077982115',
 'ours (no trace), uc30, 256-256, 0.20424979',
 'ours (no trace), uc30 0.95, 256-256, 0.225843738',
 'ours (trace), uc30, 32-256, 0.171688388',
 'ours (trace), uc30 0.95, 32-256, 0.153798021',
 'parasail, uc30, full, 0.873948339',
 'parasail, uc30 0.95, full, 1.066269822']

In [14]:
data = csv_to_pandas(output)
data

,algorithm,dataset,size,time
0,ours (no trace),uc30,32-32,0.057246
1,ours (no trace),uc30 0.95,32-32,0.063216
2,ours (no trace),uc30,32-256,0.091643
3,ours (no trace),uc30 0.95,32-256,0.077982
4,ours (no trace),uc30,256-256,0.204250
5,ours (no trace),uc30 0.95,256-256,0.225844
6,ours (trace),uc30,32-256,0.171688
7,ours (trace),uc30 0.95,32-256,0.153798
8,parasail,uc30,full,0.873948
9,parasail,uc30 0.95,full,1.066270


Uniclust30 Benchmark (AVX2)

In [15]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", axis = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = "algorithm"
).transform_filter(
    (datum.size == "32-256") | (datum.algorithm == "parasail")
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
).configure_range(
    category = {"scheme": "dark2"}
)
save(c, "uniclust30_bench.pdf")
c

alt.FacetChart(...)

Uniclust30 Block Size Benchmark (AVX2)

In [16]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", axis = None, sort = ["32-32", "32-256", "256-256"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = alt.Color("size", title = "block size", sort = ["32-32", "32-256", "256-256"])
).transform_filter(
    datum.algorithm == "ours (no trace)"
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
)
save(c, "uniclust30_size_bench.pdf")
c

alt.FacetChart(...)

## DNA Global Alignment Benchmark

In [17]:
output = !cd .. && cargo run --example nanopore_bench_global --release --features simd_avx2 --quiet
output

['# time (s)',
 'dataset, algorithm, time',
 'illumina, ours (1%-1%), 0.20235761799990465',
 'illumina, ours (1%-10%), 0.21027895599998858',
 'illumina, edlib, 0.37772163399999387',
 'illumina, ksw_extz2_sse, 0.5711356709999839',
 'illumina, wfa2, 0.0318773919999985',
 'illumina, wfa2 adaptive, 0.03170525199999888',
 'illumina, parasail, 1.9584144480000194',
 'nanopore 1kbp, ours (1%-1%), 0.2157632269999999',
 'nanopore 1kbp, ours (1%-10%), 0.266284644000001',
 'nanopore 1kbp, edlib, 0.36751824599999844',
 'nanopore 1kbp, ksw_extz2_sse, 0.5722131940000003',
 'nanopore 1kbp, wfa2, 1.3459282320000032',
 'nanopore 1kbp, wfa2 adaptive, 1.0274932389999964',
 'nanopore 1kbp, parasail, 4.145672800000009',
 'nanopore <10kbp, ours (1%-1%), 0.7686884450000007',
 'nanopore <10kbp, ours (1%-10%), 1.0920461750000008',
 'nanopore <10kbp, edlib, 1.285847981000002',
 'nanopore <10kbp, ksw_extz2_sse, 1.7109866050000029',
 'nanopore <10kbp, wfa2, 18.33174689499999',
 'nanopore <10kbp, wfa2 adaptive, 6.1

In [18]:
data = csv_to_pandas(output)
data

,dataset,algorithm,time
0,illumina,ours (1%-1%),0.202358
1,illumina,ours (1%-10%),0.210279
2,illumina,edlib,0.377722
3,illumina,ksw_extz2_sse,0.571136
4,illumina,wfa2,0.031877
5,illumina,wfa2 adaptive,0.031705
6,illumina,parasail,1.958414
7,nanopore 1kbp,ours (1%-1%),0.215763
8,nanopore 1kbp,ours (1%-10%),0.266285
9,nanopore 1kbp,edlib,0.367518


DNA Global Alignment Benchmark (AVX2)

In [19]:
algos = ["ours (1%-1%)", "ours (1%-10%)", "edlib", "ksw_extz2_sse", "wfa2", "wfa2 adaptive", "parasail"]
cs = []
for d in ["illumina", "nanopore 1kbp", "nanopore <10kbp", "nanopore <50kbp"]:
    c = alt.Chart(data.loc[data["dataset"] == d]).mark_bar().encode(
        x = alt.X("algorithm", sort = algos, title = None),
        y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
        color = alt.Color("dataset", legend = None)
    )
    t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
    c = (c + t).properties(
        width = 120,
        height = 120
    ).facet(
        column = alt.Column("dataset", title = None, header = alt.Header(orient = "top"))
    )
    cs.append(c)
c2 = (cs[0] | cs[1]) & (cs[2] | cs[3])
save(c2, "dna_global_bench.pdf")
c2

alt.VConcatChart(...)

## Nanopore Data Benchmark Setup

To run the benchmarks below, you need to clone the following repos, place them in the same directory where this repo (block aligner) is located, and follow their setup instructions:
* [diff-bench-paper](https://github.com/Daniel-Liu-c0deb0t/diff-bench-paper)
* [adaptivebandbench](https://github.com/Daniel-Liu-c0deb0t/adaptivebandbench)

## Nanopore Data Benchmark

In [20]:
output = !cd .. && cargo run --example nanopore_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, time',
 'ours (no trace 32-32), nanopore 25kbp, 1.029418487',
 'ours (no trace 32-32), random, 2.527850765',
 'ours (trace 32-32), nanopore 25kbp, 1.400272222',
 'ours (trace 32-32), random, 3.43284628',
 'ours (trace 32-64), nanopore 25kbp, 1.713347222',
 'ours (trace 32-64), random, 3.608165932']

In [21]:
data = csv_to_pandas(output)
data

,algorithm,dataset,time
0,ours (no trace 32-32),nanopore 25kbp,1.029418
1,ours (no trace 32-32),random,2.527851
2,ours (trace 32-32),nanopore 25kbp,1.400272
3,ours (trace 32-32),random,3.432846
4,ours (trace 32-64),nanopore 25kbp,1.713347
5,ours (trace 32-64),random,3.608166


In [22]:
output2 = !cd ../../diff-bench-paper/supplementary_data/benchmark_codes && ./custom_bench.sh

for i, o in enumerate(output2):
    if o.startswith("cells("):
        break
output2 = output2[i + 1:]

output2.insert(0, "algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail")
output2

['algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail',
 'editdist\t475244000\t170030000\t67475000\t712749000\t6880489\t0',
 'non-diff\t744965000\t263579000\t60931000\t1069475000\t27124786\t52',
 'diff-raw\t625991000\t208492000\t63904000\t898387000\t27291141\t32',
 'libgaba\t446407000\t155312000\t32517000\t634236000\t27121546\t53',
 'edlib\t28155033000\t19428580000\t106953000\t47690566000\t37\t0',
 'seqan\t90057570000\t0\t0\t90057570000\t0\t0']

In [23]:
data2 = csv_to_pandas(output2, d = "\t")
data2

,algorithm,fill time,trace time,convert time,total time,score,fail
0,editdist,475244000,170030000,67475000,712749000,6880489,0
1,non-diff,744965000,263579000,60931000,1069475000,27124786,52
2,diff-raw,625991000,208492000,63904000,898387000,27291141,32
3,libgaba,446407000,155312000,32517000,634236000,27121546,53
4,edlib,28155033000,19428580000,106953000,47690566000,37,0
5,seqan,90057570000,0,0,90057570000,0,0


In [24]:
cleaned2 = data2.drop(columns = ["trace time", "convert time", "total time", "score", "fail"])
cleaned2 = cleaned2.rename(columns = {"fill time": "time"})
cleaned2["time"] /= 1e9
cleaned2

,algorithm,time
0,editdist,0.475244
1,non-diff,0.744965
2,diff-raw,0.625991
3,libgaba,0.446407
4,edlib,28.155033
5,seqan,90.057570


In [25]:
cleaned = data.drop(index = [1, 3, 5])
cleaned = cleaned.drop(columns = ["dataset"])
cleaned = cleaned.append(cleaned2, ignore_index = True)
cleaned

,algorithm,time
0,ours (no trace 32-32),1.029418
1,ours (trace 32-32),1.400272
2,ours (trace 32-64),1.713347
3,editdist,0.475244
4,non-diff,0.744965
5,diff-raw,0.625991
6,libgaba,0.446407
7,edlib,28.155033
8,seqan,90.057570


25kbp Nanopore Reads Benchmark (AVX2)

In [26]:
chart1 = alt.Chart(cleaned).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm != "ours (trace 32-32)") & (datum.algorithm != "ours (no trace 32-32)") & (datum.algorithm != "ours (trace 32-64)"))

chart2 = alt.Chart(cleaned).mark_point(color = "red", filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm == "ours (trace 32-32)") | (datum.algorithm == "ours (no trace 32-32)") | (datum.algorithm == "ours (trace 32-64)"))

c = (chart1 + chart2).properties(
    width = 150,
    height = 150
)
save(c, "nanopore_bench.pdf")
c

alt.LayerChart(...)

## Sequence-to-Profile Alignment Benchmark

In [27]:
output = !cd .. && cargo run --example pssm_bench --release --features simd_avx2 --quiet
output

['size, time',
 '32-32, 0.149328607',
 '32-64, 0.174150502',
 '32-128, 0.20068917',
 '128-128, 0.202757752',
 'parasail, 0.58479644',
 '# Done!']

In [28]:
data = csv_to_pandas(output)
data

,size,time
0,32-32,0.149329
1,32-64,0.174151
2,32-128,0.200689
3,128-128,0.202758
4,parasail,0.584796


In [29]:
table = data.copy()
table = table.rename(columns = {"size": "block size"})
table["time"] = table["time"].map("{:.2}".format)
print(table)

  block size  time
0      32-32  0.15
1      32-64  0.17
2     32-128   0.2
3    128-128   0.2
4   parasail  0.58


SCOP Sequence-to-Profile Alignment Benchmark (AVX2)

In [30]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", title = "block size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"], legend = None)
).transform_filter(
    datum.size != "2048-2048"
).properties(
    width = 75,
    height = 100
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = c + t
save(c, "pssm_size_bench.pdf")
c

alt.LayerChart(...)

## WASM SIMD

[Wasmtime](https://wasmtime.dev/) is needed to run the webassembly code.

In [31]:
output = !CARGO_TARGET_WASM32_WASI_RUNNER="wasmtime --wasm-features simd --" cargo bench --target=wasm32-wasi --features simd_wasm --quiet -- --nocapture | grep 'bench:' | awk '{print $2"\t"$5}'
output

['bench_rustbio_aa_100_1000\t25,169,773',
 'bench_rustbio_aa_10_100\t268,180',
 'bench_scan_aa_1000_10000\t862,278',
 'bench_scan_aa_1000_10000_insert\t11,267,639',
 'bench_scan_aa_1000_10000_small\t603,166',
 'bench_scan_aa_1000_10000_trace\t2,023,405',
 'bench_scan_aa_100_1000\t77,946',
 'bench_scan_aa_100_1000_insert\t164,032',
 'bench_scan_aa_100_1000_small\t59,373',
 'bench_scan_aa_100_1000_trace\t225,966',
 'bench_scan_aa_10_100\t7,143',
 'bench_scan_aa_10_100_insert\t7,354',
 'bench_scan_aa_10_100_small\t5,550',
 'bench_scan_aa_10_100_trace\t88,021',
 'bench_scan_nuc_1000_10000\t576,602',
 'bench_scan_nuc_100_1000\t55,915']

In [32]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["rustbio_aa", "scan_aa", "scan_nuc"]
new_names = ["rust bio\tprotein", "ours\tprotein", "ours\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'rust bio\tprotein\t100\t1000\tdefault\t25,169,773',
 'rust bio\tprotein\t10\t100\tdefault\t268,180',
 'ours\tprotein\t1000\t10000\tdefault\t862,278',
 'ours\tprotein\t1000\t10000\tinsert\t11,267,639',
 'ours\tprotein\t1000\t10000\tsmall\t603,166',
 'ours\tprotein\t1000\t10000\ttrace\t2,023,405',
 'ours\tprotein\t100\t1000\tdefault\t77,946',
 'ours\tprotein\t100\t1000\tinsert\t164,032',
 'ours\tprotein\t100\t1000\tsmall\t59,373',
 'ours\tprotein\t100\t1000\ttrace\t225,966',
 'ours\tprotein\t10\t100\tdefault\t7,143',
 'ours\tprotein\t10\t100\tinsert\t7,354',
 'ours\tprotein\t10\t100\tsmall\t5,550',
 'ours\tprotein\t10\t100\ttrace\t88,021',
 'ours\tnucleotide\t1000\t10000\tdefault\t576,602',
 'ours\tnucleotide\t100\t1000\tdefault\t55,915']

In [33]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,rust bio,protein,100,1000,default,25169773
1,rust bio,protein,10,100,default,268180
2,ours,protein,1000,10000,default,862278
3,ours,protein,1000,10000,insert,11267639
4,ours,protein,1000,10000,small,603166
5,ours,protein,1000,10000,trace,2023405
6,ours,protein,100,1000,default,77946
7,ours,protein,100,1000,insert,164032
8,ours,protein,100,1000,small,59373
9,ours,protein,100,1000,trace,225966


In [34]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (WASM SIMD)

In [35]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench_wasm.pdf")
c

alt.Chart(...)